<a href="https://colab.research.google.com/github/kisung2577/CMPE-255/blob/main/Assignment_5_for_nearest_neighbor_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Using state of art libraries for Approximate nearest neighbor search




##INSTALLING NECESSARY LIBRARIES

In [1]:
!pip install lightfm
!pip install nmslib
!pip install faiss
!pip install annoy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 5.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705372 sha256=0f64d1996484df7369cfb3d32f2179b0f6984b43c26a53e31cb12ebfe3231ddd
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 5.0 MB/s 
     |████████████████████████████████| 188 kB 42.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 4.9 MB/s 
  Created wheel for 

In [2]:
!pip install faiss-cpu --no-cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 5.2 MB/s 


##IMPORTING NECESSARY LIBRARIES

In [4]:
!pip install stackapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stackapi: filename=StackAPI-0.2.0-py3-none-any.whl size=5859 sha256=dcdca1b53f50cd7d0f0bef6f545df3d404b29ab0a9b7808d2d42cbb8e3695eec
  Stored in directory: /root/.cache/pip/wheels/ec/db/60/df42a65853e3581c26a2fbb2012a228cb8e267369a3b9ca44d
Successfully built stackapi


In [5]:
import pickle
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import nmslib
import faiss
import annoy
import time
from stackapi import StackAPI

##LOAD DATA

In [7]:
stackexchange = fetch_stackexchange('crossvalidated')
stackexchange

{'train': <2836x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 51477 stored elements in COOrdinate format>,
 'test': <2836x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 8020 stored elements in COOrdinate format>,
 'item_features': <72360x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 72360 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17')}

In [8]:
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

In [9]:
stackexchange['item_feature_labels']

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

###Pickle data

In [10]:
with open('stack_exchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

###Load data and visualize it

In [11]:
def load_data():
    with open('stack_exchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[ 0.00697032,  0.04790997,  0.12536475, ..., -0.12463009,
         -0.05288965,  0.13861355],
        [-0.02087457, -0.02841803,  0.13443491, ..., -0.11852854,
         -0.0703564 , -0.12902519],
        [ 0.010651  ,  0.0415263 ,  0.12601689, ..., -0.1496781 ,
         -0.13284197, -0.14351618],
        ...,
        [ 0.01942083, -0.03132042, -0.00682115, ...,  0.02231003,
          0.02628561, -0.02916586],
        [-0.01936831, -0.02114354,  0.04661608, ..., -0.00233051,
         -0.00840903, -0.01128837],
        [-0.00639899,  0.06478348,  0.08963732, ...,  0.04589099,
         -0.00139314, -0.00056165]], dtype=float32)}

##LSH
Locality-sensitive hashing (LSH) is an algorithmic technique that hashes similar input items into the same "buckets" with high probability. The number of buckets is much smaller than the universe of possible input items. Since similar items end up in the same buckets, this technique can be used for data clustering and nearest neighbor search. It differs from conventional hashing techniques in that hash collisions are maximized, not minimized. 

###VISUALIZING VECTORS

In [12]:
vectors_value = data["vector"]
names_value = data["name"]

In [13]:
vectors_value

array([[ 0.00697032,  0.04790997,  0.12536475, ..., -0.12463009,
        -0.05288965,  0.13861355],
       [-0.02087457, -0.02841803,  0.13443491, ..., -0.11852854,
        -0.0703564 , -0.12902519],
       [ 0.010651  ,  0.0415263 ,  0.12601689, ..., -0.1496781 ,
        -0.13284197, -0.14351618],
       ...,
       [ 0.01942083, -0.03132042, -0.00682115, ...,  0.02231003,
         0.02628561, -0.02916586],
       [-0.01936831, -0.02114354,  0.04661608, ..., -0.00233051,
        -0.00840903, -0.01128837],
       [-0.00639899,  0.06478348,  0.08963732, ...,  0.04589099,
        -0.00139314, -0.00056165]], dtype=float32)

In [14]:
names_value

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

###Creating Index Class

In [15]:
class LSHIndex():
  def __init__(self, vectors, labels):
      self.dimension = vectors.shape[1]
      self.vectors = vectors.astype('float32')
      self.labels = labels
  def build_index(self, num_bits=8):
      self.index = faiss.IndexLSH(self.dimension, num_bits)
      self.index.add(self.vectors)
  def results(self, vectors, k=10):
      distances, indices = self.index.search(vectors, k) 
      return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [16]:
time_1 = time.time()

lsh_ind = LSHIndex(data["vector"], data["name"])
lsh_ind.build_index()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(lsh_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("LSH took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:12
 question_id:14
 question_id:36
 question_id:63
 question_id:70
 question_id:126
 question_id:141
 question_id:173
 question_id:280
LSH took 0.03609466552734375 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [17]:
site_name = 'stackoverflow'
site_value = StackAPI(site_name)
question = site_value.fetch('questions/{ids}', ids=[337], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'page': 1,
 'quota_max': 300,
 'quota_remaining': 299,
 'total': 0,
 'items': [{'tags': ['python', 'xml'],
   'owner': {'reputation': 3644,
    'user_id': 111,
    'user_type': 'registered',
    'accept_rate': 73,
    'profile_image': 'https://www.gravatar.com/avatar/0b4711b9840dae35e1f921354dc320c0?s=256&d=identicon&r=PG',
    'display_name': 'saalon',
    'link': 'https://stackoverflow.com/users/111/saalon'},
   'is_answered': True,
   'view_count': 10153,
   'protected_date': 1376504248,
   'accepted_answer_id': 342,
   'answer_count': 12,
   'score': 82,
   'last_activity_date': 1636743381,
   'creation_date': 1217648155,
   'last_edit_date': 1483102581,
   'question_id': 337,
   'content_license': 'CC BY-SA 3.0',
   'link': 'https://stackoverflow.com/questions/337/xml-processing-in-python',
   'title': 'XML Processing in Python',
   'body': "<p>I am about to build a piece of a project that will need to construct and post an XML document to a web

Question with question id - 6

In [18]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


Question with question id - 1010

In [19]:
print("Following question is similar:\n" + question['items'][0]['title'])


Following question is similar:
XML Processing in Python


##Exhaustive Search
Any search process in which every item of a set is checked before a decision is made about the presence or absence of a target item. This may be a search for particular items in memory, a visual search, or any problem-solving exercise that involves choosing the best path to a solution from a number of alternative paths.

###Creating Index Class

In [20]:
class ExhaustiveSearchIndex():
  def __init__(self, vectors, labels):
    self.dimension = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  def build_index(self):
    self.index = faiss.IndexFlatL2(self.dimension,)
    self.index.add(self.vectors)

  def results(self, vectors, k=10):
    distances, indices = self.index.search(vectors, k) 
    return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [21]:
time_1 = time.time()

exi_ind = ExhaustiveSearchIndex(data["vector"], data["name"])
exi_ind.build_index()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(exi_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("Exhaustive Search took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:9
 question_id:64302
 question_id:70779
 question_id:37425
 question_id:28595
 question_id:375
 question_id:27131
 question_id:22895
 question_id:21656
Exhaustive Search took 0.053157806396484375 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [22]:
question = site_value.fetch('questions/{ids}', ids=[9], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'page': 1,
 'quota_max': 300,
 'quota_remaining': 298,
 'total': 0,
 'items': [{'tags': ['c#', '.net', 'datetime'],
   'owner': {'reputation': 62831,
    'user_id': 1,
    'user_type': 'registered',
    'accept_rate': 100,
    'profile_image': 'https://www.gravatar.com/avatar/51d623f33f8b83095db84ff35e15dbe8?s=256&d=identicon&r=PG',
    'display_name': 'Jeff Atwood',
    'link': 'https://stackoverflow.com/users/1/jeff-atwood'},
   'is_answered': True,
   'view_count': 768713,
   'accepted_answer_id': 1404,
   'answer_count': 71,
   'community_owned_date': 1313523643,
   'score': 2176,
   'last_activity_date': 1659461135,
   'creation_date': 1217547659,
   'last_edit_date': 1658961276,
   'question_id': 9,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/9/how-do-i-calculate-someones-age-based-on-a-datetime-type-birthday',
   'title': 'How do I calculate someone&#39;s age based on a DateTime type birthday?',
   'bo

Question for question id 6

In [23]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


In [24]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
How do I calculate someone&#39;s age based on a DateTime type birthday?


##Trees and Graphs

###Create Index Class

In [25]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def results(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [26]:
time_1 = time.time()

annoy_ind = AnnoyIndex(data["vector"], data["name"])
annoy_ind.build()

vector, name = data['vector'][6], data['name'][6]

simlar_question_return_ids = '\n '.join(annoy_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("Trees and Graphs took {total_time} seconds.".format(total_time=total_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:64302
 question_id:715
 question_id:629
 question_id:54
 question_id:207
 question_id:1771
 question_id:141
 question_id:583
 question_id:126
Trees and Graphs took 0.945528507232666 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [27]:
question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'page': 1,
 'quota_max': 300,
 'quota_remaining': 294,
 'total': 0,
 'items': [{'tags': ['html', 'css', 'internet-explorer-7'],
   'owner': {'reputation': 24972,
    'user_id': 9,
    'user_type': 'registered',
    'accept_rate': 32,
    'profile_image': 'https://www.gravatar.com/avatar/23c1e2063688620b75b248e08c0d5c24?s=256&d=identicon&r=PG',
    'display_name': 'Kevin Dente',
    'link': 'https://stackoverflow.com/users/9/kevin-dente'},
   'is_answered': True,
   'view_count': 23219,
   'protected_date': 1414626308,
   'accepted_answer_id': 31,
   'answer_count': 7,
   'score': 319,
   'last_activity_date': 1611946005,
   'creation_date': 1217542088,
   'last_edit_date': 1611946005,
   'question_id': 6,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/6/why-did-the-width-collapse-in-the-percentage-width-child-element-in-an-absolutel',
   'title': 'Why did the width collapse in the percentage width child element 

Question for question id 6

In [28]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


Similar question

In [29]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


##Product Quantization
Product quantization (PQ) is an effective vector quantization method. A product quantizer can generate an exponentially large codebook at very low memory/time cost. The essence of PQ is to decompose the high-dimensional vector space into the Cartesian product of subspaces and then quantize these subspaces separately.

###Create Index Class

In [30]:
class PQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
        
    def build(self, 
              number_of_partition=8, 
              search_in_x_partitions=2, 
              subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def results(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [31]:
time_1 = time.time()

annoy_ind = PQIndex(data["vector"], data["name"])
annoy_ind.build()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(annoy_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("Product Quantization took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:916
 question_id:375
 question_id:23824
 question_id:9184
 question_id:40542
 question_id:788
 question_id:11863
 question_id:13887
 question_id:63256
Product Quantization took 3.110276460647583 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [32]:
question = site_value.fetch('questions/{ids}', ids=[145], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'page': 1,
 'quota_max': 300,
 'quota_remaining': 292,
 'total': 0,
 'items': [{'tags': ['c#', '.net', 'file', 'compression'],
   'owner': {'reputation': 2285,
    'user_id': 87,
    'user_type': 'registered',
    'accept_rate': 67,
    'profile_image': 'https://www.gravatar.com/avatar/1f8377ba373028db1c2598a963c9ee2c?s=256&d=identicon&r=PG',
    'display_name': 'DylanJ',
    'link': 'https://stackoverflow.com/users/87/dylanj'},
   'is_answered': True,
   'view_count': 5217,
   'answer_count': 9,
   'community_owned_date': 1396616569,
   'score': 61,
   'last_activity_date': 1584530660,
   'creation_date': 1217610788,
   'last_edit_date': 1545747183,
   'question_id': 145,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/145/compressing-decompressing-folders-files',
   'title': 'Compressing / Decompressing Folders &amp; Files',
   'body': '<p>Does anyone know of a good way to compress or decompress files and folde

Question with question id 6

In [33]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


In [34]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
Compressing / Decompressing Folders &amp; Files


##HNSW
Hierarchical Navigable Small World (HNSW) graphs are among the top-performing indexes for vector similarity search[1]. HNSW is a hugely popular technology that time and time again produces state-of-the-art performance with super fast search speeds and fantastic recall.

Yet despite being a popular and robust algorithm for approximate nearest neighbors (ANN) searches, understanding how it works is far from easy.

###Create Class Index

In [35]:
class NMSLIBIndex():
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  def build(self):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.vectors)
    self.index.createIndex({'post': 2})
        
  def results(self, vector, k=10):
    indices = self.index.knnQuery(vector, k=k)
    return [self.labels[i] for i in indices[0]]

###Checking the time taken for the algorithm to return the data
Here we are checking for the similar questions to question id 6

In [36]:
time_1 = time.time()

hnsw_ind = NMSLIBIndex(data["vector"], data["name"])
hnsw_ind.build()

vector, name = data['vector'][6:7], data['name'][6]

simlar_question_return_ids = '\n '.join(hnsw_ind.results(vector))
print(f"Similar questions to question id - {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_1

print("HNSW took {total_time} seconds.".format(total_time=total_time))

Similar questions to question id - question_id:6 are:
 question_id:6
 question_id:9
 question_id:138
 question_id:90
 question_id:50
 question_id:22
 question_id:151
 question_id:24
 question_id:28
 question_id:375
HNSW took 82.95296955108643 seconds.


###Let us retrieve the question id for one of the questions from the above returned result along with the original question

In [37]:
question = site_value.fetch('questions/{ids}', ids=[90], filter='withbody')
question

{'backoff': 0,
 'has_more': False,
 'page': 1,
 'quota_max': 300,
 'quota_remaining': 290,
 'total': 0,
 'items': [{'tags': ['svn', 'tortoisesvn', 'branch', 'branching-and-merging'],
   'owner': {'reputation': 46687,
    'user_id': 58,
    'user_type': 'registered',
    'accept_rate': 91,
    'profile_image': 'https://www.gravatar.com/avatar/298e0497aa6b76a573f17e6a2bb22dec?s=256&d=identicon&r=PG',
    'display_name': 'cmcculloh',
    'link': 'https://stackoverflow.com/users/58/cmcculloh'},
   'is_answered': True,
   'view_count': 56308,
   'closed_date': 1356493549,
   'accepted_answer_id': 1466832,
   'answer_count': 3,
   'score': 164,
   'last_activity_date': 1503594083,
   'creation_date': 1217601684,
   'last_edit_date': 1503594083,
   'question_id': 90,
   'link': 'https://stackoverflow.com/questions/90/how-do-you-branch-and-merge-with-tortoisesvn',
   'closed_reason': 'not a real question',
   'title': 'How do you branch and merge with TortoiseSVN?',
   'body': '<p>How do you <

Question id - 6

In [38]:
asked_question = site_value.fetch('questions/{ids}', ids=[6], filter='withbody')
print("Question Id 6\n" + asked_question['items'][0]['title'])

Question Id 6
Why did the width collapse in the percentage width child element in an absolutely positioned parent on Internet Explorer 7?


Similar Question

In [39]:
print("Following question is similar:\n" + question['items'][0]['title'])

Following question is similar:
How do you branch and merge with TortoiseSVN?
